In [1]:
import os
from os import listdir
from os.path import isfile,join
from datetime import datetime,timedelta
import requests
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from math import sqrt 
from collections import Counter
from datetime import date, datetime, timedelta
import time

/Users/nadiaaitmada/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# I. Automatiser le téléchargement des fichiers de données
# But : Créer un script qui télécharge les fichiers au fil des jours et 
#       évite de télécharger à nouveau les fichiers déjà téléchargés
#       (Fichiers publiés chaque jour)

# Inconvénient : nécessite de créer un script de job en local pour faire tourner cahque jour  
#               Si un fichier n'est pas récupérer le jour même, il ne peut pas être récupérer le jour suivant

## Pour info : Chemin du répertoire d'enregistrement des répertoires (identique au notebook) 
working_directory = os.getcwd()
print(f"Chemin d'accès au répertoire : {working_directory}")

# Création d'un dossier pour stocker les fichiers téléchargés
os.makedirs("sauvegarde_log",exist_ok=True)
os.makedirs("sauvegarde_degra",exist_ok=True)

Chemin d'accès au répertoire : /Users/nadiaaitmada/Desktop/Wild/projet3/Automatisation


In [32]:
# Fonction pour obtenir la date d'aujourd'hui
def get_today():
    today = datetime.today()
    return today.strftime("%Y-%m-%d")

# Récupérer année/mois/jour actuel
current_year = datetime.now().year
current_month = datetime.now().month
current_day = datetime.now().day

# Utiliser la fonction today
today = get_today()

# Construction de l'URL de téléchargement en fonction du mois et du jour
for year in range(current_year, current_year+1):
    for month in range(current_month, current_month+1):
        for day in range(current_day, current_day+1):
            if year == current_year and month == current_month and day > current_day:
                continue
            if f"{year}-{month:02d}-{day:02d}" < today:
                continue

            url_log_vol = f"http://sc-e.fr/docs/logs_vols_2024-{month:02d}-{day:02d}.csv"
            file_log = f"sauvegarde_log/logs_vols_2024-{month:02d}-{day:02d}.csv"

            # Télécharger le fichier
            print(f"Téléchargement de {file_log}")

            response = requests.get(url_log_vol)

            # Si réponse 200, on réalise le téléchargement
            if response.status_code == 200:
                with open(file_log, "wb") as file:
                    file.write(response.content)
                print(f"Téléchargement de {file_log} terminé avec succès.")
            else:
                print(f"Téléchargement impossible de {file_log}. Code statut {response.status_code}")

print("Téléchargement terminé")

# Lire le fichier de log téléchargé
df_log = pd.read_csv(file_log)

# Lien vers le fichier des aéronefs
link_aero = '/Users/nadiaaitmada/Desktop/Wild/projet3/Automatisation/aeronef_nettoye.csv'
aeronefs = pd.read_csv(link_aero)

# Mise à jour des colonnes en fonction des données du fichier de log
dico_voyant = {}
for i in range(1, 4):
    dico_voyant[i] = df_log.loc[df_log['etat_voyant'] == i, 'aero_linked'].to_list()

jour_vol = df_log.loc[df_log['etat_voyant'] != 0, 'jour_vol'].max()
jour_vol = datetime.strptime(jour_vol, '%Y-%m-%d')

for index, row in aeronefs.iterrows():
    if row['ref_aero'] in dico_voyant[1]:
        aeronefs.at[index, 'en_maintenance'] = True
        aeronefs.at[index, 'last_maint'] = jour_vol.strftime('%Y-%m-%d')
        aeronefs.at[index, 'end_maint'] = (jour_vol + timedelta(days=1)).strftime('%Y-%m-%d')
        aeronefs.at[index, 'cout'] = 1 * 15000
    elif row['ref_aero'] in dico_voyant[2]:
        aeronefs.at[index, 'en_maintenance'] = True
        aeronefs.at[index, 'last_maint'] = jour_vol.strftime('%Y-%m-%d')
        aeronefs.at[index, 'end_maint'] = (jour_vol + timedelta(days=7)).strftime('%Y-%m-%d')
        aeronefs.at[index, 'cout'] = 7 * 15000
    elif row['ref_aero'] in dico_voyant[3]:
        aeronefs.at[index, 'en_maintenance'] = True
        aeronefs.at[index, 'last_maint'] = jour_vol.strftime('%Y-%m-%d')
        aeronefs.at[index, 'end_maint'] = (jour_vol + timedelta(days=14)).strftime('%Y-%m-%d')
        aeronefs.at[index, 'cout'] = 14 * 15000

# Sauvegarder les modifications dans le fichier CSV
aeronefs.to_csv(link_aero, index=False)
print("Mise à jour du fichier aeronef terminée.")

        

Téléchargement de sauvegarde_log/logs_vols_2024-07-23.csv
Téléchargement de sauvegarde_log/logs_vols_2024-07-23.csv terminé avec succès.
Téléchargement terminé
Mise à jour du fichier aeronef terminée.


In [4]:
# Construction de l'url de téléchargement en fonction du mois et du jour
for year in range(current_year,current_year+1):
    for month in range(current_month,current_month+1):
        for day in range(current_day,current_day+1):
            if year == current_year and month == current_month and day > current_day: # Ignorer les fichiers supérieurs à la date d'aujourd'hui
                continue
            if f"{year}-{month:02d}-{day:02d}" < today:   # Ignorer les fichiers déjà téléchargés
                continue

            url_degrad = f"http://sc-e.fr/docs/degradations_2024-{month:02d}-{day:02d}.csv"
            file_degrad = f"sauvegarde_degra/degradations_2024-{month:02d}-{day:02d}.csv"

# Télécharger le fichier
            print(f"Téléchargement de {file_degrad}")

            response = requests.get(url_degrad)

# Si réponse 200, on réalise le téléchargement
            if response.status_code == 200:
                
                with open(file_degrad,"wb") as file_degradation:
                    file_degradation.write(response.content)
            else : 
                print(f"Téléchargement impossible de {file_degrad}. Code statut {response.status_code}")
print("Téléchargement terminé")


# Lire le fichier de log téléchargé
df_degra = pd.read_csv(file_degrad)

# Lien vers le fichier des composants
link_compo = '/Users/nadiaaitmada/Desktop/Wild/projet3/Automatisation/composant_nettoye.csv'
composants = pd.read_csv(link_compo)

#a partir du fichier degra daily qui contient que les dégradations du jour - créer un dictionnaire avec les colo,nnes 'compo_concerned' et 'tx_usure_new'
dico_degra_day = dict(zip(df_degra['compo_concerned'], df_degra['usure_nouvelle']))

#Itérer sur la paire clé / valeur du dictionnaire des dégradations du jour
# si la clé = la ref du composant est égale à une des valeurs de la colonne ref composant de la BDD COMPOSANTS
#alors on remplace la valeur de la cellule 'taux_usure_actuelle' de la même ligne par la valeur du dico

for key, value in dico_degra_day.items():
    composants.loc[composants['ref_compo'] == str(key), 'taux_usure_actuel'] = value

# Sauvegarder les modifications dans le fichier CSV
composants.to_csv(link_compo, index=False)
print("Mise à jour du fichier composant terminée.")

Téléchargement de sauvegarde_degra/degradations_2024-07-23.csv
Téléchargement terminé


In [ ]:
######################################################################################## VÉRIFICATION DE L'IMPORT DES FICHIERS         A SUPPRIMER ENSUITE #######################################################

In [7]:
# Vérifier le nombre de fichiers téléchargés dans sauvegarde_log
log_path = "./sauvegarde_log"
listfile_log = [file for file in listdir(log_path) if isfile(join(log_path,file))]
print(f" Nb fichier log_vol",len(listfile_log))

degra_path = "./sauvegarde_degra"
listfile_degra = [file for file in listdir(degra_path) if isfile(join(degra_path,file))]
print(f" Nb fichier degradation",len(listfile_degra))

 Nb fichier log_vol 1
 Nb fichier degradation 1


In [10]:
# Vérifier l'ouverture du fichier log_vol téléchargé
link_log = '/Users/nadiaaitmada/Desktop/Wild/projet3/Automatisation/sauvegarde_log/logs_vols_2024-07-23.csv'
df_log = pd.read_csv(link_log)
df_log.head()

,ref_vol,aero_linked,jour_vol,time_en_air,sensor_data,etat_voyant
0,V09194923,E170_0081,2024-07-23,6.6,"{'temp': '-10.0°C', 'pressure': '932.0 hPa', '...",0
1,V07241227,B747_6747,2024-07-23,6.6,"{'temp': '1.1°C', 'pressure': '963.7 hPa', 'vi...",0
2,V01443548,A321_5206,2024-07-23,4.3,"{'temp': '-14.2°C', 'pressure': '969.5 hPa', '...",1
3,V06352606,B747_4152,2024-07-23,5.1,"{'temp': '12.7°C', 'pressure': '956.8 hPa', 'v...",0
4,V04714204,B747_0472,2024-07-23,7.3,"{'temp': '-3.7°C', 'pressure': '928.2 hPa', 'v...",0


In [11]:
# Vérifier l'ouverture d'un fichier degradation
link_degra = '/Users/nadiaaitmada/Desktop/Wild/projet3/Automatisation/sauvegarde_degra/degradations_2024-07-23.csv'
df_degra = pd.read_csv(link_degra)
df_degra.head()

,ref_deg,linked_aero,compo_concerned,usure_nouvelle,measure_day,need_replacement
0,D003661,E170_6353,REAE170-E170_6353-0,74.467365,2024-07-22,False
1,D009402,E170_6353,REAE170-E170_6353-1,72.213322,2024-07-22,False
2,D000812,E170_6353,SYSE170-E170_6353-2,33.828077,2024-07-22,False
3,D006665,E170_6353,ORDE170-E170_6353-3,23.588377,2024-07-22,False
4,D001163,E170_6353,AUTE170-E170_6353-4,66.115359,2024-07-22,False


In [ ]:
######################################################################################## FIN VÉRIFICATION DE L'IMPORT DES FICHIERS         #######################################################

In [ ]:
####################### Vérifier si la modification à bien eu lieu dasn aeronef###########

In [42]:
# Ouverture du fichier aeronef nettoyé
link_aero = '/Users/nadiaaitmada/Desktop/Wild/projet3/Automatisation/aeronef_nettoye.csv'
aeronefs = pd.read_csv(link_aero)
print(aeronefs.shape)

#check si la boucle à fonctionner
aeronefs.loc[aeronefs['ref_aero'] == 'A321_5206', :]

(231, 7)


,ref_aero,type_model,debut_service,last_maint,en_maintenance,end_maint,cout
78,A321_5206,A321,2018-06-07,2024-07-23,True,2024-07-24,15000.0


In [43]:
# Aeronef d'origine 

link_aero2 = '/Users/nadiaaitmada/Desktop/Wild/projet3/Automatisation/aeronef_nettoye copie.csv'
aero_copie = pd.read_csv(link_aero2)
print(aero_copie.shape)

#check si la boucle à fonctionner
aero_copie.loc[aero_copie['ref_aero'] == 'A321_5206', :]

(231, 6)


,ref_aero,type_model,debut_service,last_maint,en_maintenance,end_maint
78,A321_5206,A321,2018-06-07,2024-06-01,False,NaN


In [ ]:
################################################################################################ VERIFIER SI COMPOSANT A ETE MIS A JOUR###

In [49]:
# Ouverture du fichier aeronef nettoyé
link_compo = '/Users/nadiaaitmada/Desktop/Wild/projet3/Automatisation/composant_nettoye.csv'
composants = pd.read_csv(link_compo)
print(composants.shape)

#check si la boucle à fonctionner
composants.loc[composants['ref_compo'] == 'HUBCRJ700-CRJ700_2863-42', :]

(10395, 7)


,ref_compo,categorie,aero,desc,lifespan,taux_usure_actuel,cout
2721,HUBCRJ700-CRJ700_2863-42,Composants Secondaires,CRJ700_2863,Hublots,68978,20.417488,3593


In [50]:
link_compo2 = '/Users/nadiaaitmada/Desktop/Wild/projet3/Automatisation/composant_nettoye copie.csv'
compo_copie = pd.read_csv(link_compo2)
print(compo_copie.shape)

#check si la boucle à fonctionner
compo_copie.loc[compo_copie['ref_compo'] == 'HUBCRJ700-CRJ700_2863-42', :]

(10395, 7)


,ref_compo,categorie,aero,desc,lifespan,taux_usure_actuel,cout
2721,HUBCRJ700-CRJ700_2863-42,Composants Secondaires,CRJ700_2863,Hublots,68978,29.455472,3593
